In [1]:
import pandas as pd
import numpy as np
from collections import Counter
# import eli5

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import LinearSVC, NuSVC

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, Dropout, Flatten
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50

import csv

# from sklearn.decomposition import PCA

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train.csv')

train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9850 entries, 0 to 9849
Data columns (total 2 columns):
Image    9850 non-null object
Id       9850 non-null object
dtypes: object(2)
memory usage: 154.0+ KB


,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


In [3]:
classes = list(train_df['Id'].unique())
num_features = len(classes)

In [4]:
pretrained_model = Sequential()
pretrained_model.add(ResNet50(include_top=False, weights='imagenet', pooling='avg'))
pretrained_model.add(Dense(num_features, activation='softmax'))

pretrained_model.layers[0].trainable = False

pretrained_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

pretrained_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 4251)              8710299   
Total params: 32,298,011
Trainable params: 8,710,299
Non-trainable params: 23,587,712
_________________________________________________________________


In [5]:
idg_final = ImageDataGenerator(
    data_format='channels_last',
    rescale=1./255,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    rotation_range=15,
)

traing_gen = idg_final.flow_from_directory('./train', classes=classes, target_size=(224, 224), class_mode='categorical')                  

pretrained_model.fit_generator(traing_gen, epochs=10, verbose=1)

pretrained_model.save('final_model')

Found 9850 images belonging to 4251 classes.
Epoch 1/10
308/308 [==============================] - 3985s 13s/step - loss: 8.9218 - acc: 0.0860
Epoch 2/10
308/308 [==============================] - 3555s 12s/step - loss: 3.2710 - acc: 0.3403
Epoch 3/10
308/308 [==============================] - 3594s 12s/step - loss: 1.8597 - acc: 0.5836
Epoch 4/10
308/308 [==============================] - 3656s 12s/step - loss: 1.2712 - acc: 0.7058
Epoch 5/10
308/308 [==============================] - 3667s 12s/step - loss: 0.9556 - acc: 0.7795
Epoch 6/10
308/308 [==============================] - 3689s 12s/step - loss: 0.7665 - acc: 0.8207
Epoch 7/10
308/308 [==============================] - 3693s 12s/step - loss: 0.6581 - acc: 0.8498
Epoch 8/10
308/308 [==============================] - 3618s 12s/step - loss: 0.5874 - acc: 0.8636
Epoch 9/10
308/308 [==============================] - 3823s 12s/step - loss: 0.5144 - acc: 0.8797
Epoch 10/10
308/308 [==============================] - 4334s 14s/step - l

In [6]:
pretrained_model = load_model('final_model')

In [10]:
# Ignore this, this is to predict on the test set but I'm not going 
# to worry about this until I can get reasonable results on the training set.

idg_testing = ImageDataGenerator(
    rescale=1./255,
    data_format='channels_last', 
)

testing_gen = idg_testing.flow_from_directory('./test', target_size=(224, 224), class_mode=None, shuffle=False)
predictions = pretrained_model.predict_generator(testing_gen, verbose=1)

Found 15610 images belonging to 1 classes.
488/488 [==============================] - 6753s 14s/step


In [19]:
# This debug code is for evaluating on the training set.

import operator

idg_debug = ImageDataGenerator(
    data_format='channels_last',
    rescale=1./255,
)

other_classes = [c for (c, _) in sorted(traing_gen.class_indices.items(), key=operator.itemgetter(1))]

debug_gen = idg_debug.flow_from_directory('./train', classes=classes, target_size=(224, 224), class_mode='categorical')
print(pretrained_model.evaluate_generator(debug_gen))

True


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [11]:
def decode_predictions(preds, class_list, top=5):
    decoded = []
    
    for pred in preds:        
        ind = pred.argsort()[-top:][::-1]

        class_arr = np.array(class_list)
        decoded.append(class_arr[ind])
    
    return decoded

decoded = decode_predictions(predictions, top=5, class_list=classes)

assert len(decoded) == len(testing_gen.filenames)

with open('predictions.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Image', 'Id'])
    
    for fname, preds in zip(testing_gen.filenames, decoded):
        image = fname.split('/')[1]
        writer.writerow([image, ' '.join(preds)])

In [ ]:
# model = Sequential()

# model.add(Conv2D(32, kernel_size=(3,3), input_shape=(256,256,1)))
# model.add(MaxPool2D(pool_size=(2,2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dense(num_features, activation='softmax'))

# model.compile(
#     optimizer='sgd',
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )

In [7]:
# import os
# import shutil

# for idx, image, ident in train_df.itertuples():
#     label_directory = './train/{}'.format(ident)
#     if not os.path.isdir(label_directory):
#         os.makedirs(label_directory)
    
#     shutil.move('./train/{}'.format(image), label_directory)

In [13]:
# TODO: add more data augmentation
# idg_training = ImageDataGenerator(
#     data_format='channels_last', 
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     rotation_range=45,
#     zca_whitening=True, 
#     validation_split=0.1 
# )

# traing_gen = idg_training.flow_from_directory('./train', subset='training', classes=classes, target_size=(224, 224), class_mode='categorical')
# validation_gen = idg_training.flow_from_directory('./train', subset='validation', classes=classes, target_size=(224, 224), class_mode='categorical')

# pretrained_model.fit_generator(traing_gen, epochs=1, verbose=1, validation_data=validation_gen)
# print(pretrained_model.model.metrics_names, pretrained_model.evaluate_generator(validation_gen))

# pretrained_model.save('training_model')

Found 9692 images belonging to 4251 classes.
Found 158 images belonging to 4251 classes.
Epoch 1/1
303/303 [==============================] - 5938s 20s/step - loss: 8.0261 - acc: 0.0750 - val_loss: 4.2759 - val_acc: 0.5127
['loss', 'acc'] [4.275886239884775, 0.512658228979835]
